In [ ]:
import pandas as pd
import os

file_path = r"C:\Users\hp\Desktop\lab_isotope\20240705OC"
csv_name = file_path.split("\\")[5]
# print(csv_name)
file_list = os.listdir(file_path)
file_list = sorted(file_list,key = lambda x: (x[:8], int(x[18:-4]) if x[18:-4].isdigit() else float('inf')))

# 创建一个空的数据列表
data_list = []
# L = list(range(1,260))
for file_name in file_list:
    # folder_name = f"Sediment 20231202_{i}.raw"
    folder_path = os.path.join(r"C:\Users\hp\Desktop\lab_isotope\20240705OC", file_name)  # 文件夹路径
    file_path_C = os.path.join(
        folder_path, "Results for CO2 by CF (uncalibrated)(Fn3).xls"
    )  # 文件路径
    file_path_N = os.path.join(
        folder_path, "Results for N2 by CF (uncalibrated)(Fn2).xls"
    )

    dataf = pd.read_excel(file_path_C, sheet_name="Printout", header=40, converters={"Major Height (nA)": str})
    dataf["Major Height (nA)"] = dataf["Major Height (nA)"].apply(pd.to_numeric, errors='coerce')

    dataf_N = pd.read_excel(file_path_N, sheet_name="Printout", header=40, converters={"Major Height (nA)": str})
    dataf_N["Major Height (nA)"] = dataf_N["Major Height (nA)"].apply(pd.to_numeric, errors='coerce')


    # print(dataf.columns)
    # print(dataf.index)

    C_Major_Height = dataf["Major Height (nA)"].max()
    C_max_row_index =  dataf["Major Height (nA)"].idxmax()
    delta_values = dataf["delta13C"][C_max_row_index]

    N_major_height = dataf_N["Major Height (nA)"].max()
    N_max_row_index =  dataf_N["Major Height (nA)"].idxmax()
    delta_values_N = dataf_N["deltaN15"][N_max_row_index]

    data_list.append(
        {
            "File": file_name,
            "delta13C": delta_values,
            "C Major Height (nA)": C_Major_Height,
            "deltaN15": delta_values_N,
            "N Major Height (nA)": N_major_height,
        }
    )

# # 创建结果 DataFrame
dataframe = pd.DataFrame(data_list)

# # 将结果保存为 CSV 文件
dataframe.to_csv(f'sample_C_data_{csv_name}.csv', index=False)

# # 打印所有元素
# display(dataframe)

In [ ]:
import os
import pandas as pd


for i in range(4,8):
    folder_name = f"sulf 20231026_{i}.raw"
    folder_path = os.path.join(r"D:\Lab_data\20231025\20231025", folder_name)  # 文件夹路径
    file_path_C = os.path.join(
        folder_path, "Results for CO2 by CF (uncalibrated)(Fn3).xls"
    )  # 文件路径
    file_path_N = os.path.join(
        folder_path, "Results for N2 by CF (uncalibrated)(Fn2).xls"
    )

    dataf = pd.read_excel(file_path_C, sheet_name="Printout", header=40)

    dataf_N = pd.read_excel(file_path_N, sheet_name="Printout", header=40)

    print(dataf.columns)
    print(dataf.index)

    delta_values = dataf.loc[1]["delta13C"]

    C_Major_Height = dataf.loc[1]["Major Height (nA)"]

    delta_values_N = dataf_N.loc[1]["deltaN15"]

    N_major_height = dataf_N.loc[1]["Major Height (nA)"]

    data_list.append(
        {
            "File": folder_name,
            "delta13C": delta_values,
            "C Major Height (nA)": C_Major_Height,
            "deltaN15": delta_values_N,
            "N Major Height (nA)": N_major_height,
        }
    )


# # 创建结果 DataFrame
dataframe = pd.DataFrame(data_list)

# # 将结果保存为 CSV 文件
dataframe.to_csv('sulf_CO2_data.csv', index=False)

# # 打印所有元素
display(dataframe)

In [ ]:
from glob import glob
import pandas as pd
import re


def read_c(file):
    dataf = pd.read_excel(file, sheet_name="Printout", header=40)
    dataf = dataf.iloc[[1]][["Major Height (nA)", "delta13C"]]
    dataf["File"] = file.split("\\")[0]
    dataf["idx"] = int(re.split("\W+|_", file)[2])
    dataf = dataf.rename(columns={"Major Height (nA)": "C Major Height (nA)"})
    return dataf


def read_n(file):
    dataf = pd.read_excel(file, sheet_name="Printout", header=40)
    dataf = dataf.iloc[[1]][["Major Height (nA)", "deltaN15"]]
    dataf["File"] = file.split("\\")[0]
    dataf["idx"] = int(re.split("\W+|_", file)[2])
    dataf = dataf.rename(columns={"Major Height (nA)": "N Major Height (nA)"})
    return dataf


co2files = glob("Sediment*/Results for CO2 by CF (uncalibrated)(Fn3).xls")
co2sulf = glob("sulf*/Results for CO2 by CF (uncalibrated)(Fn3).xls")
N2files = glob("Sediment*/Results for N2 by CF (uncalibrated)(Fn2).xls")
N2sulf = glob("sulf*/Results for N2 by CF (uncalibrated)(Fn2).xls")

df_c = pd.concat([read_c(file) for file in co2files])
df_c = df_c.set_index("idx").sort_index()
df_n = pd.concat([read_n(file) for file in N2files])
df_n = df_n.set_index("idx").sort_index()


df_csulf = pd.concat([read_c(file) for file in co2sulf])
df_csulf["idx"] = range(1, len(co2sulf) + 1)
df_csulf["idx"] = len(df_c) // len(df_csulf) * df_csulf["idx"] + 1
df_csulf = df_csulf.set_index("idx").sort_index()

df_Nsulf = pd.concat([read_n(file) for file in N2sulf])
df_Nsulf["idx"] = range(1, len(N2sulf) + 1)
df_Nsulf["idx"] = len(df_n) // len(df_Nsulf) * df_Nsulf["idx"] + 1
df_Nsulf = df_Nsulf.set_index("idx").sort_index()


display(df_c)
display(df_csulf)
display(df_n)
display(df_Nsulf)

display(
    pd.concat(
        [
            df_c,
            df_csulf,
        ]
    ).sort_index()
)
display(pd.concat([df_c,df_csulf,df_n,df_Nsulf]).sort_index())

# # 创建结果 DataFrame

dataframe = pd.DataFrame(pd.concat([df_c,df_csulf,df_n,df_Nsulf]).sort_index())

# # 将结果保存为 CSV 文件
dataframe.to_csv("sample_data__1.csv", index=False)

# # 打印所有元素
display(dataframe)

In [ ]:
import os

file_path = r"C:\Users\user\Desktop\lab_data\20231202OC"
file_list = os.listdir(file_path)
file_list = sorted(file_list,key = lambda x: (x[:8], int(x[18:-4]) if x[18:-4].isdigit() else float('inf')))
display(file_list)